# Ignore all the bs below, used it to build my stochastic models. I'll let you know when the important stuff begins

In [3]:
for month in ["jan18","feb18", "mar18", "may18"]
    data1 = CSV.read(month*"_filter_agg_scen1.csv")
    data2 = CSV.read(month*"_filter_agg_scen2.csv")
    data3 = CSV.read(month*"_filter_agg_scen3.csv")
    data4 = CSV.read(month*"_filter_agg_scen4.csv")
    s1 = size(unique(data1[!,:start_area]),1)
    println(s1)
    s2 = size(unique(data2[!,:start_area]),1)
    println(s2)
    s3 = size(unique(data3[!,:start_area]),1)
    println(s3)
    s4 = size(unique(data4[!,:start_area]),1)
    e1 = size(unique(data1[!,:end_area]),1)
    e2 = size(unique(data1[!,:end_area]),1)
    e3 = size(unique(data1[!,:end_area]),1)
    e4 = size(unique(data1[!,:end_area]),1)
    if s1 == s2 & s1 == s3 & s1 == s4 & s1 == e1 & s1 == e2 & s1 == e3 & s1 == e4
        println(month*" ALL GOOD")
    else
        println(month*" NOT GOOD")
    end
end

50
50
50
jan18 ALL GOOD
50
50
50
feb18 ALL GOOD
50
50
50
mar18 ALL GOOD
50
50
50
may18 ALL GOOD


In [10]:
data1 = CSV.read("may19_flows.csv")
names(data1)

10-element Array{Symbol,1}:
 :Column1              
 :Day                  
 :Hour                 
 :start_time_interval  
 :start_area           
 :end_area             
 :dep_flow             
 :hourly_dep_flow_total
 :hourly_dep_flow_pct  
 :allowed_flows        

In [17]:
unique(data1[!,:Hour])

24-element Array{Int64,1}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23

In [15]:
start_stations = Set(unique(data1[!,2]))
end_stations = Set(unique(data1[!,3]))
stations = collect(union(start_stations,end_stations))
time_intervals = unique(data1[!,8])
days = unique(data1[!,7]);

In [16]:
#define sizes 
# m = total number of start station clusters
m= size(stations,1)
# n = total number of time windows
n = size(time_intervals,1)
# o = total number of days
o = size(days,1);


In [17]:
r_ride = 1.15
c_bike = 345;

# Optimization Model

In [20]:
function optimize_distribution(data)
    #initialize model
    mod = Model(solver = GurobiSolver())

    #decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o] >= 0,Int)

    #objective function
    @objective(mod,Max,sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o))


    #constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o],x[i,t,l] - sum(d[i,j,t,l] for j=1:m) + sum(d[j,i,t,l] for j=1:m)-x[i,t+1,l]==0)
    @constraint(mod,[d=1:o], sum(x[i,1,d] for i=1:m) == sum(x[i,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]] <= data[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o],d[i,i,t,l]==0)
    @constraint(mod,r_ride*sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o) - c_bike*sum(x[i,1,1] for i=1:m) >= -200000)



    #solve model
    status = solve(mod)
    
    return sum(getvalue(x)[:,1,1]),getvalue(d),getobjectivevalue(mod)
end



optimize_distribution (generic function with 1 method)

In [21]:
optimize_distribution(data1)

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1870531 rows, 1836000 columns and 7088950 nonzeros
Model fingerprint: 0x8997650e
Variable types: 36000 continuous, 1800000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 1463.0000000
Presolve removed 1852443 rows and 1746517 columns (presolve time = 5s) ...
Presolve removed 1852443 rows and 1746528 columns
Presolve time: 8.84s
Presolved: 18088 rows, 89472 columns, 249573 nonzeros
Variable types: 18058 continuous, 71414 integer (53421 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       

(564.0, [-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

...

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

...

[-0.0 1.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 

# Baseline Model

In [75]:
function baseline_distribution(min_bikes,data)
    uniform_dist = round(min_bikes/m)
    mod = Model(solver = GurobiSolver())

    #decision variables
    @variable(mod,x[i=1:m,t=1:n,l=1:o]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o] >= 0)

    #objective function
    @objective(mod,Max,sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o))


    #constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o],x[i,t,l] - sum(d[i,j,t,l] for j=1:m) + sum(d[j,i,t,l] for j=1:m)-x[i,t+1,l]==0)
    @constraint(mod,[d=1:o], sum(x[i,1,d] for i=1:m) == sum(x[i,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]] <= data[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o],d[i,i,t,l]==0)
    @constraint(mod,[i=1:m,d=1:o],x[i,1,d]==uniform_dist)
#     @constraint(mod,r_ride*sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o) - c_bike*sum(x[i,1,1] for i=1:m) >= -200000)

    status = solve(mod)
    
    return getvalue(d)
end


baseline_distribution (generic function with 1 method)

# Everything Above is Useless, The Real Shit Starts from Here

Instructions to run this baby:
1. Make sure your files are named properly and that you have the csv files for all 4 scenarios for the months you want to run the model for. The file naming convention is "mmmmyy_filter_agg_scenx.csv" .For example, "sept19_filter_agg_scen1.csv"
2. Every csv file should have the following columns with this exact naming convention: "start_area", "end_area", "dep_flow","Hour"(0-23),"Day"(1-30/31).
3. Run the functions below and then call the compute_everything function. The arguments to this function are 
    - a list of months you want to compute everything. For example - ["sept19","may18","oct20"]. Every month in this list should match the way the month is spelt in the dataset. 
    - The acceptable total loss per month (eg 200000)
    - Operational Revenue - Operational costs (we discussed this number to be 1.15)
    - Fixed cost of a bike (we discussed this to be 200)
    - example call of the function - main_result = compute_everything(["sept19"],200000,1.15,200)
4. The function ouputs the following:
    - A CSV file with the optimal number of bikes per station per day for each month in the list of months
    - A CSV file with the optimal model allowed flows for Scenario 1.
    - A dataframe with the following information for each month (each row of the dataframe corresponds to one month in the list of months: Minimum Number of Bikes from the optimal model, Number of missed rides for the optimal and baseline scenarios, number of met rides for the optimal and baseline scenarios.

In [1]:
using JuMP, Gurobi, DataFrames, CSV, Plots

# Stochastic Model

In [2]:
function compute_missed_rides(data,d)
    missed_rides = [(data[i,:dep_flow]-d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]])
    for i=1:size(data,1)]
    return sum(missed_rides)
end

compute_missed_rides (generic function with 1 method)

In [3]:
#### stochastic optimization model
function stochastic_model(data1,data2,data3,data4,acceptable_loss,r_ride,c_bike)
    
    # m = total number of start station clusters
    m= size(unique(data1[!,:start_area]),1)
    # n = total number of time windows
    n = size(unique(data1[!,:Hour]),1)
    # o = total number of days
    o = size(unique(data1[!,:Day]),1)
    
    #1. initialize model
    
    mod = Model(solver = GurobiSolver())
    
    #2. decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o,s=1:4]>= 0,Int)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o,s=1:4] >= 0,Int)
    @variable(mod,max_stat,Int)
    
    #3. objective function
    @objective(mod,Max,(sum(d[i,j,t,l,s] for i=1:m,j=1:m,t=1:n,l=1:o,s=1:4)/4) - 0.0001*max_stat)
    
    
    #4. constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o,s=1:4],x[i,t,l,s] - sum(d[i,j,t,l,s] for j=1:m) + sum(d[j,i,t,l,s] for j=1:m)-x[i,t+1,l,s]==0)
    @constraint(mod,[d=1:o,s=1:4], sum(x[i,1,d,s] for i=1:m) == sum(x[i,1,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data1,1)],d[data1[i,:start_area],data1[i,:end_area],data1[i,:Hour]+1,data1[i,:Day],1] <= data1[i,:dep_flow])
    @constraint(mod,[i=1:size(data2,1)],d[data2[i,:start_area],data2[i,:end_area],data2[i,:Hour]+1,data2[i,:Day],2] <= data2[i,:dep_flow])
    @constraint(mod,[i=1:size(data3,1)],d[data3[i,:start_area],data3[i,:end_area],data3[i,:Hour]+1,data3[i,:Day],3] <= data3[i,:dep_flow])
    @constraint(mod,[i=1:size(data4,1)],d[data4[i,:start_area],data4[i,:end_area],data4[i,:Hour]+1,data4[i,:Day],4] <= data4[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o,s=1:4],d[i,i,t,l,s]==0)
    @constraint(mod,r_ride*(sum(d[i,j,t,l,s] for i=1:m,j=1:m,t=1:n,l=1:o,s=1:4)/4) - c_bike*sum(x[i,1,1,1] for i=1:m) >= -acceptable_loss)
    @constraint(mod,[i=1:m,d=1:o,s=1:4], max_stat >= x[i,1,d,s])
    
    #5. solve model
    status = solve(mod)
    
    #6. Compute missed rides and average it for the 4 scenarios
    missed_rides1 = compute_missed_rides(data1,getvalue(d)[:,:,:,:,1])
    missed_rides2 = compute_missed_rides(data2,getvalue(d)[:,:,:,:,2])
    missed_rides3 = compute_missed_rides(data3,getvalue(d)[:,:,:,:,3])
    missed_rides4 = compute_missed_rides(data4,getvalue(d)[:,:,:,:,4])
    avg_missed_rides = (missed_rides1+missed_rides2+missed_rides3+missed_rides4)/4
    
    #7. Compute number of rides met and average it for each scenario
    avg_met_rides = getobjectivevalue(mod)
    
    #8. Input number of bikes per station per day into a dataframe (only for scenario 1)
    bikes = getvalue(x)[:,:,:,1]
    x_df = DataFrame(Station=Int[],Hour=Int[],Day=Int[],Num_Bikes=Int[])
    for i=1:m
        for j=1:n
            for k=1:o
                push!(x_df,[i,j,k,bikes[i,j,k]])
            end
        end
    end
    
    
    #9. Input allowed flows from scenario 1 into its dataframe (data1)
    flows= getvalue(d)[:,:,:,:,1]
    allowed_flows = []
    for i in 1:size(data1,1)
        append!(allowed_flows,flows[data1[i,:start_area],data1[i,:end_area],data1[i,:Hour]+1,data1[i,:Day]])
    end
    data1[:allowed_flows] = allowed_flows
    final_flows_data = data1[:,:]
    
   
    
    #10. return
    return sum(getvalue(x)[:,1,1,1]),avg_missed_rides,avg_met_rides,x_df,final_flows_data
    
end
    
    
    
    
    
    
    
    
    

stochastic_model (generic function with 1 method)

# Stochastic Baseline 

In [4]:
function stochastic_baseline(data1,data2,data3,data4,min_bikes,r_ride,c_bike)
    # m = total number of start station clusters
    m= size(unique(data1[!,:start_area]),1)
    # n = total number of time windows
    n = size(unique(data1[!,:Hour]),1)
    # o = total number of days
    o = size(unique(data1[!,:Day]),1)
    
    #1. uniform_dist = uniformly distributed number of bikes at each station
    uniform_dist = round(min_bikes/m)
    
    #2. initialize model
    
    mod = Model(solver = GurobiSolver())
    
    #3. decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o,s=1:4]>= 0,Int)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o,s=1:4] >= 0,Int)
    
    #4. objective function
    @objective(mod,Max,sum(d[i,j,t,l,s] for i=1:m,j=1:m,t=1:n,l=1:o,s=1:4)/4)
    
    
    #5. constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o,s=1:4],x[i,t,l,s] - sum(d[i,j,t,l,s] for j=1:m) + sum(d[j,i,t,l,s] for j=1:m)-x[i,t+1,l,s]==0)
    @constraint(mod,[d=1:o,s=1:4], sum(x[i,1,d,s] for i=1:m) == sum(x[i,1,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data1,1)],d[data1[i,:start_area],data1[i,:end_area],data1[i,:Hour]+1,data1[i,:Day],1] <= data1[i,:dep_flow])
    @constraint(mod,[i=1:size(data2,1)],d[data2[i,:start_area],data2[i,:end_area],data2[i,:Hour]+1,data2[i,:Day],2] <= data2[i,:dep_flow])
    @constraint(mod,[i=1:size(data3,1)],d[data3[i,:start_area],data3[i,:end_area],data3[i,:Hour]+1,data3[i,:Day],3] <= data3[i,:dep_flow])
    @constraint(mod,[i=1:size(data4,1)],d[data4[i,:start_area],data4[i,:end_area],data4[i,:Hour]+1,data4[i,:Day],4] <= data4[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o,s=1:4],d[i,i,t,l,s]==0)
    @constraint(mod,[i=1:m,d=1:o,s=1:4],x[i,1,d,s]==uniform_dist)
    
    #6. solve model
    status = solve(mod)
    
    #7. Compute missed rides and average it for the 4 scenarios
    missed_rides1 = compute_missed_rides(data1,getvalue(d)[:,:,:,:,1])
    missed_rides2 = compute_missed_rides(data2,getvalue(d)[:,:,:,:,2])
    missed_rides3 = compute_missed_rides(data3,getvalue(d)[:,:,:,:,3])
    missed_rides4 = compute_missed_rides(data4,getvalue(d)[:,:,:,:,4])
    avg_missed_rides = (missed_rides1+missed_rides2+missed_rides3+missed_rides4)/4
    
    #8. Compute number of rides met and average it for each scenario
    avg_met_rides = getobjectivevalue(mod)
   
    
    #9. return
    return avg_missed_rides,avg_met_rides
    
end
    

stochastic_baseline (generic function with 1 method)

# Main Function That Computes Everything

In [5]:
function compute_everything(month_list,acceptable_loss,r_ride,c_bike)
    main_df = DataFrame(Month=String[],Min_Bikes=Int[],Missed_Optimal=Int[],
    Missed_Baseline=Int[],Met_Optimal=Int[],Met_Baseline=Int[])
    for i in month_list
        
        #read in datasets for the 4 scenarios
        
        data1 = CSV.read("$i"*"_filter_agg_scen1.csv")
        data2 = CSV.read("$i"*"_filter_agg_scen2.csv")
        data3 = CSV.read("$i"*"_filter_agg_scen3.csv")
        data4 = CSV.read("$i"*"_filter_agg_scen4.csv")
        
        #define sizes - do it for one dataset because its the same for all 4 scenarios
        
        # m = total number of start station clusters
        m= size(unique(data1[!,:start_area]),1)
        # n = total number of time windows
        n = size(unique(data1[!,:Hour]),1)
        # o = total number of days
        o = size(unique(data1[!,:Day]),1)
        
        #run optimal model and get outputs
        optimal_solution = stochastic_model(data1,data2,data3,data4,acceptable_loss,r_ride,c_bike)
        Min_Bikes = optimal_solution[1]
        Missed_Optimal = optimal_solution[2]
        Met_Optimal = optimal_solution[3]
        CSV.write("$i"*"_bike_distribution.csv",optimal_solution[4])
#         CSV.write("$i"*"_flows.csv",optimal_solution[5])
        
        #run baseline model and get outputs
#         baseline_solution = stochastic_baseline(data1,data2,data3,data4,Min_Bikes,r_ride,c_bike)
#         Missed_Baseline = baseline_solution[1]
#         Met_Baseline = baseline_solution[2]
        
        
#         #push values into main dataframe
#         push!(main_df,[i,round(Min_Bikes),
#                 round(Missed_Optimal),round(Missed_Baseline),round(Met_Optimal),round(Met_Baseline)])
        
    end
    
    return main_df
end
    

compute_everything (generic function with 1 method)

In [6]:
main_result = compute_everything(["sep19"],200000,1.15,200)

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7488121 rows, 7344001 columns and 28367950 nonzeros
Model fingerprint: 0x276f7072
Variable types: 0 continuous, 7344001 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+02]
  Objective range  [1e-04, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 1286.5000000
Presolve removed 1 rows and 0 columns (presolve time = 6s) ...
Presolve removed 7347008 rows and 150000 columns (presolve time = 13s) ...
Presolve removed 7348862 rows and 6959653 columns (presolve time = 19s) ...
Presolve removed 7390195 rows and 7000986 columns (presolve time = 21s) ...
Presolve removed 7396842 rows and 7010000 columns (presolve time = 26s) ...
Presolve removed 7414945 rows and 7028374 columns (presolve time = 30s) ...
Presolve

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = stochastic_model(::DataFrame, ::DataFrame, ::DataFrame, ::DataFrame, ::Int64, ::Float64, ::Int64) at In[3]:66
└ @ Main ./In[3]:66


,Month,Min_Bikes,Missed_Optimal,Missed_Baseline,Met_Optimal,Met_Baseline
,String,Int64,Int64,Int64,Int64,Int64
